In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import zenhan
from jeraconv import jeraconv
j2w = jeraconv.J2W()

import subprocess
import os

import requests
from bs4 import BeautifulSoup
import re

In [2]:
resposne = requests.get("https://www.gyoukaku.go.jp/hotline/index.html")
soup = BeautifulSoup(resposne.text, 'html.parser')
pattern = re.compile("\.xlsx$")

URL_LIST = []
for link in soup.find_all('a'):
	if pattern.search(link.get('href')):
		URL_LIST.append(
			os.path.join(
				"./rawdata/",
				os.path.basename(link.get('href'))
			)
		)

FILE_LIST = []
ITEM_NUM = len(URL_LIST)//2

for i in range(0, ITEM_NUM):
	FILE_LIST.append(
		{
			"type": "規制改革" if i < ITEM_NUM //2 else "行政改革",
			"teian": URL_LIST[i],
			"kaitou": URL_LIST[i+ITEM_NUM],
			"year": int(re.search("R([0-9]+)\.", URL_LIST[i]).group(1)),
			"headoffset_teian": 3 if i < ITEM_NUM//2 else 1,
			"headoffset_kaitou":2
		}
	)
	if i >= ITEM_NUM//2 and FILE_LIST[-1]["year"] == 2:
		FILE_LIST[-1]["headoffset_teian"] = 2
print(FILE_LIST)

JSON_SUFFIX="_processed"

[{'type': '規制改革', 'teian': './rawdata/02_teian_kisei_20220921_R4.xlsx', 'kaitou': './rawdata/14_kaitou_kisei_20220921_R4.xlsx', 'year': 4, 'headoffset_teian': 3, 'headoffset_kaitou': 2}, {'type': '規制改革', 'teian': './rawdata/04_teian_kisei_20220921_R3.xlsx', 'kaitou': './rawdata/16_kaitou_kisei_20220921_R3.xlsx', 'year': 3, 'headoffset_teian': 3, 'headoffset_kaitou': 2}, {'type': '規制改革', 'teian': './rawdata/06_teian_kisei_20220921_R2.xlsx', 'kaitou': './rawdata/18_kaitou_kisei_20220921_R2.xlsx', 'year': 2, 'headoffset_teian': 3, 'headoffset_kaitou': 2}, {'type': '行政改革', 'teian': './rawdata/08_teian_gyoukaku_20220921_R4.xlsx', 'kaitou': './rawdata/20_kaitou_gyoukaku_20220921_R4.xlsx', 'year': 4, 'headoffset_teian': 1, 'headoffset_kaitou': 2}, {'type': '行政改革', 'teian': './rawdata/10_teian_gyoukaku_20220921_R3.xlsx', 'kaitou': './rawdata/22_kaitou_gyoukaku_20220921_R3.xlsx', 'year': 3, 'headoffset_teian': 1, 'headoffset_kaitou': 2}, {'type': '行政改革', 'teian': './rawdata/12_teian_gyoukaku_20

In [3]:
baseurl = "https://www.gyoukaku.go.jp/hotline/img/"
for filepath in FILE_LIST:
    if not os.path.exists(filepath['teian']):
        url = baseurl + os.path.basename(filepath['teian'])
        subprocess.call(['wget', '--directory-prefix', './rawdata', url], shell=True)

    if not os.path.exists(filepath['kaitou']):
        url = baseurl + os.path.basename(filepath['kaitou'])
        subprocess.call(['wget', '--directory-prefix', './rawdata', url], shell=True)

## Helper関数

In [4]:
import datetime

## Cases
# STRING: '令和\n3年'
# INT: EXCEL DATE
# DATETIME: datetime

def normalize(data):
    if isinstance(data, int) or isinstance(data, float):
        return pd.to_datetime('1900-01-01') + pd.to_timedelta(data, 'D')
    elif isinstance(data, datetime.datetime):
        return data
    elif isinstance(data, str):
        oneline = zenhan.z2h(data.replace('\n', ''))
        Y, MD = oneline.split('年')
        try:
            Y = str(j2w.convert(Y + '年')) + '年'
        except:
            Y = str(j2w.convert('令和' + Y + '年')) + '年'
        date = datetime.datetime.strptime(Y+MD, '%Y年%m月%d日')
        return date
    else:
        return data

def conv_date(df, column):
    tmp = df[column].apply(normalize)
    return tmp

## 本題

In [5]:
df_list = []
for item in FILE_LIST:
	print(item["teian"], item["headoffset_teian"])
	df = pd.read_excel(item["teian"], header=item["headoffset_teian"])
	df.rename(columns={
		'所管省庁への　　検討要請日': '所管省庁への検討要請日',
		'所管省庁への\n検討要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討\n要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討要請日': '所管省庁への検討要請日',
		'内閣府での\n回答取りまとめ\n日':'回答取りまとめ日',
		'内閣府での\n回答取りまとめ日':'回答取りまとめ日',
		'提案事項名\n（タイトル）':'提案事項',
		'提案主体名\n（会社名・団体名）':'提案主体',
		'制度の所管省庁': '所管省庁',
		'ワーキング・グループにおける\n処理方針': 'ワーキング・グループにおける処理方針'
	}, inplace=True)
	print(df.columns)
	df['所管省庁への検討要請日'] = conv_date(df, '所管省庁への検討要請日')
	df['回答取りまとめ日'] = conv_date(df, '回答取りまとめ日')

	print(item["kaitou"])
	df2 = pd.read_excel(item["kaitou"], header=[0, 1])
	df2.rename(columns={
		'所管省庁への\n検討要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討\n要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討要請日': '所管省庁への検討要請日',
		'内閣府での\n回答取りまとめ\n日':'回答取りまとめ日',
		'提案事項名\n（タイトル）':'提案事項名',
		'提案主体名\n（会社名・団体名）':'提案主体名',
		'ワーキング・グループにおける\n処理方針': 'ワーキング・グループにおける処理方針'
	}, inplace=True)
	df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]
	df2['所管省庁への検討要請日'] = conv_date(df2, '所管省庁への検討要請日')
	df2['回答取りまとめ日'] = conv_date(df2, '回答取りまとめ日')

	df3 = pd.merge(df, df2, how='left', on='番号')
	df3.drop(
		columns=[
			'提案事項_y', '提案主体_y', '所管省庁_y', '備考', '備考_x', '備考_y', 'Unnamed: 7',
			'ワーキング・グループにおける処理方針', 'ワーキング・グループにおける処理方針_x', 'ワーキング・グループにおける処理方針_y'],
		inplace = True, errors='ignore')
	def a(x):
		try:
			tmp = x['所管省庁への検討要請日_y']
			if tmp is pd.NaT:
				tmp = x['所管省庁への検討要請日_x']
		except:
			print(x)
		return tmp
	df3['所管省庁への検討要請日_x'] = df3.apply(a, raw=False, axis=1)

	def b(x):
		try:
			tmp = x['回答取りまとめ日_x']
			if tmp == pd.NaT:
				tmp = x['回答取りまとめ日_y']
		except:
			print(x)
		return tmp
	df3['回答取りまとめ日_x'] = df3.apply(b, raw=False, axis=1)
	df3= df3.drop(columns=['所管省庁への検討要請日_y', '回答取りまとめ日_y'])

	df3.rename(columns={
		'所管省庁への検討要請日_x': '所管省庁への検討要請日',
		'回答取りまとめ日_x': '回答取りまとめ日',
		'提案事項_x': '提案事項',
		'提案主体_x': '提案主体',
		'所管省庁_x': '所管省庁',
		'所管省庁の検討結果_対応の\n分類': '所管省庁の検討結果_対応の分類'
	}
	, inplace=True)

	df3.dropna(subset=['番号'], inplace=True)
	df3['番号'] = df3['番号'].astype(int)
	df3['区分'] = item['type']
	df3['通番'] = item['type'][0] + str(item['year']) + df3['番号'].map('-{:04d}'.format)
	print(df3.columns)

	df_list.append(df3.copy())

df_all = pd.concat(df_list, ignore_index=True)

./rawdata/02_teian_kisei_20220921_R4.xlsx 3
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考',
       'ワーキング・グループにおける処理方針'],
      dtype='object')
./rawdata/14_kaitou_kisei_20220921_R4.xlsx
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '受付番号', '受付日',
       '提案の具体的内容', '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等',
       '所管省庁の検討結果_対応の分類', '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/04_teian_kisei_20220921_R3.xlsx 3


C:\Users\tsasa\AppData\Local\Temp\ipykernel_3976\3681273349.py:32: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考',
       'ワーキング・グループにおける処理方針'],
      dtype='object')
./rawdata/16_kaitou_kisei_20220921_R3.xlsx


C:\Users\tsasa\AppData\Local\Temp\ipykernel_3976\3681273349.py:32: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/06_teian_kisei_20220921_R2.xlsx 3
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考',
       'ワーキング・グループにおける処理方針'],
      dtype='object')
./rawdata/18_kaitou_kisei_20220921_R2.xlsx


...

Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/10_teian_gyoukaku_20220921_R3.xlsx 1
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考'], dtype='object')
./rawdata/22_kaitou_gyoukaku_20220921_R3.xlsx


C:\Users\tsasa\AppData\Local\Temp\ipykernel_3976\3681273349.py:32: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/12_teian_gyoukaku_20220725_R2.xlsx 2
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考'], dtype='object')
./rawdata/24_kaitou_gyoukaku_20220725_R2.xlsx


C:\Users\tsasa\AppData\Local\Temp\ipykernel_3976\3681273349.py:32: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')


In [6]:
df_all = df_all.loc[:, ~ df_all.columns.str.contains('^Unnamed')]

In [7]:
df_all

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項,提案主体,所管省庁,受付番号,受付日,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,区分,通番
0,1,2022-06-29,2022-07-22,障害者雇用の法定雇用率を身体・知的・精神の3障害それぞれに設定する,個人,厚生労働省,NaN,NaN,現在の法定雇用率は3障害合算での数字のため、障害程度が健常者とほとんど変わりない軽度の障害者...,うつ病などの精神疾患により精神障害者保健福祉手帳を所持していると障害者雇用の法定雇用率算定対...,○法定雇用率\n全ての事業主は、従業員の一定割合（法定雇用率）以上の障害者を雇用することが義...,障害者の雇用の促進等に関する法律第43条第２項、第43条第４項,対応不可,○法定雇用率制度は、「採用の自由」に一定の制約を加えるものであることから、その制約は必要かつ...,規制改革,規4-0001
1,2,2022-06-29,2022-07-22,調理師試験は実務経験なしにて受験資格を与える。,個人,厚生労働省,NaN,NaN,今調理師試験は 実務経験２年で受験資格を与えている。実務経験なしで試験に合格しその後実務経験...,福祉施設や病院などの厨房（ちゅうぼう）では、直営でも委託の場合でも、人手が足りていません。な...,○調理師法で、「調理師」とは、調理師の名称を用いて調理の業務に従事することができる者として都...,調理師法第２条\n調理師法第３条,対応不可,調理師免許は、左記のとおり、「調理、栄養及び衛生に関して調理師たるに必要な知識及び技能」を有...,規制改革,規4-0002
2,3,2022-06-29,NaT,登記完了証の記載事項を、書面／オンライン問わず、登記記録を転写すること,商業登記ゲンロン,法務省,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規4-0003
3,4,2022-06-29,2022-07-22,特定社会保険労務士の業務,個人,厚生労働省,NaN,NaN,規制改革・民間開放推進3か年計画に基づき、特定社会保険労務士の業務に関して\n１．個別労働紛...,日々の労務管理、労働相談に従事している特定社会保険労務士が労働紛争に携わることは有益なことで...,特定社会保険労務士は、個別労働紛争について厚生労働大臣が指定する団体が行う裁判外紛争解決手...,社会保険労務士法第２条及び同法第２条の２,対応不可,平成26年の社会保険労務士法改正により、社会保険労務士の業務の範囲が広がったところであり、...,規制改革,規4-0004
4,5,2022-06-29,2022-07-22,弁護士等による戸籍謄本等の職務上請求書について、職印の押印を廃止すること,個人,法務省,NaN,NaN,弁護士等による戸籍謄本等の職務上請求書について、職印の押印を必要とする戸籍法施行規則の規定を...,近年の押印廃止の流れにより、一般国民や官公署が戸籍謄本を請求する場合は押印を要しなくなったが...,弁護士等が、戸籍法第１０条の２第３項から第５項までの規定に基づき戸籍謄本等を請求する場合には...,戸籍法第１０条の２第３項～第５項\n戸籍法第１０条の３第１項\n戸籍法施行規則第１１条の２第４号,対応不可,統一請求書の使用及び当該統一請求書に係る弁護士等の職印の押印については、不正請求を防止する観...,規制改革,規4-0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3789,1030,2021-08-29,2021-11-06,政府統計のローデータをSQLにて公開する,個人,総務省,NaN,NaN,\n政府統計全般のローデータを（個人情報保護の処理は行ったうえで）SQLサーバにて公開し、広...,\n現在の政府統計は形式がバラバラであるが、もしこれを仮にCSV等の形式としてe-Stadで...,統計を作成するために用いられる調査票情報（ローデータ）については、個人又は法人その他の団体の...,なし,現行制度下で対応可能,制度の現状欄に記載のとおりです。,行政改革,行2-1030
3790,1031,2021-08-29,NaT,文書作成時、スペースを使った均等割り付けの禁止,個人,デジタル庁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行2-1031
3791,1032,2021-08-29,NaT,国家資格の免許証のカード化,個人,デジタル庁\n総務省\n警察庁\n金融庁\n消費者庁\n法務省\n財務省\n文部科学省\n厚...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行2-1032
3792,1033,2021-09-04,2021-11-06,土地に関して、様々な組織が管理する情報を統合して管理すべき（縦割りの解消）,個人,国土交通省\n法務省,NaN,NaN,\n土地の現状に関して主に公的機関が関する情報（登記に関する地図（公図）、国土地理院の地図、...,\n土地の権利の確認は、まず登記記録の確認から始まる。公図は不整合のものが大変多く、実際の土...,【法務省】\n各官署において，所管する制度等に応じ，必要な情報を保有しているものと承知してい...,測量法第27条第2項、道路法第28条、土地収用法第47条の2第1項、第66条第3項、下水道法...,対応不可,【法務省】\n 制度の現状欄に記載のとおり，各官署において所管する制度等に応じて当該制度に係...,行政改革,行2-1033


## タグ付処理

In [8]:
import MeCab
tagger = MeCab.Tagger("-Ochasen -u ./dict/local_dict.dic")

In [9]:
def get_word_bag(text):
    try:
        tag = tagger.parseToNode(text)
        words = set()
        while tag:
            feature = tag.feature.split(",")
            if feature[0] == "名詞" and len(tag.surface) >= 2:
                words.add(tag.surface)
            tag = tag.next
        return list(words)
    except:
        return []

In [10]:
all_text = df_all['提案事項'] + \
           df_all['提案の具体的内容'].replace(np.nan, '', regex=True) + \
           df_all['提案理由'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_制度の現状'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_対応の概要'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_該当法令等'].replace(np.nan, '', regex=True)
tmp = all_text.apply(get_word_bag)

In [11]:
import json

with open('./dict/tag.json', encoding='utf-8') as json_file:
    rules_json = json.load(json_file)

def get_summary_tag(words):
    tags = []
    rules = rules_json['rules']

    for rule in rules:
        if any(item in words for item in rule['def']):
            tags.append(rule['tag'])
    return '\n'.join(map(str,tags))

In [12]:
df_all['タグ'] = tmp.apply(get_summary_tag)

## 過去ログ参照

In [13]:
df_all.loc[df_all['所管省庁の検討結果_制度の現状'].str.match('^(番号|規制改革|行政改革)', na=False), '所管省庁の検討結果_対応の分類'] = '回答済'

## データ保存

In [14]:
# 概要
df_all['UID'] = df_all.index.map('UID{:05d}'.format)
df_all.drop_duplicates(subset=['通番'], inplace=True)
df_all[['UID','番号', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '所管省庁への検討要請日','所管省庁の検討結果_対応の分類',
        '区分', '通番','タグ']].to_json('./s3static/src/assets/data{}.json'.format(JSON_SUFFIX), orient='records', force_ascii=False)
# 詳細
df_detailed = df_all[['UID','提案の具体的内容', '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等',
       '所管省庁の検討結果_対応の分類', '所管省庁の検討結果_対応の概要']]
df_detailed = df_detailed.set_index('UID')
df_detailed.dropna(subset=['提案の具体的内容'], inplace=True) 
df_detailed.to_json('./s3static/public/data/data{}.json'.format(JSON_SUFFIX), orient='index', force_ascii=False)

In [15]:
df_all.columns

Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '受付番号', '受付日',
       '提案の具体的内容', '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等',
       '所管省庁の検討結果_対応の分類', '所管省庁の検討結果_対応の概要', '区分', '通番', 'タグ', 'UID'],
      dtype='object')

In [16]:
df_all

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項,提案主体,所管省庁,受付番号,受付日,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,区分,通番,タグ,UID
0,1,2022-06-29,2022-07-22,障害者雇用の法定雇用率を身体・知的・精神の3障害それぞれに設定する,個人,厚生労働省,NaN,NaN,現在の法定雇用率は3障害合算での数字のため、障害程度が健常者とほとんど変わりない軽度の障害者...,うつ病などの精神疾患により精神障害者保健福祉手帳を所持していると障害者雇用の法定雇用率算定対...,○法定雇用率\n全ての事業主は、従業員の一定割合（法定雇用率）以上の障害者を雇用することが義...,障害者の雇用の促進等に関する法律第43条第２項、第43条第４項,対応不可,○法定雇用率制度は、「採用の自由」に一定の制約を加えるものであることから、その制約は必要かつ...,規制改革,規4-0001,,UID00000
1,2,2022-06-29,2022-07-22,調理師試験は実務経験なしにて受験資格を与える。,個人,厚生労働省,NaN,NaN,今調理師試験は 実務経験２年で受験資格を与えている。実務経験なしで試験に合格しその後実務経験...,福祉施設や病院などの厨房（ちゅうぼう）では、直営でも委託の場合でも、人手が足りていません。な...,○調理師法で、「調理師」とは、調理師の名称を用いて調理の業務に従事することができる者として都...,調理師法第２条\n調理師法第３条,対応不可,調理師免許は、左記のとおり、「調理、栄養及び衛生に関して調理師たるに必要な知識及び技能」を有...,規制改革,規4-0002,教育\n医療\nコロナ,UID00001
2,3,2022-06-29,NaT,登記完了証の記載事項を、書面／オンライン問わず、登記記録を転写すること,商業登記ゲンロン,法務省,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規4-0003,デジタル,UID00002
3,4,2022-06-29,2022-07-22,特定社会保険労務士の業務,個人,厚生労働省,NaN,NaN,規制改革・民間開放推進3か年計画に基づき、特定社会保険労務士の業務に関して\n１．個別労働紛...,日々の労務管理、労働相談に従事している特定社会保険労務士が労働紛争に携わることは有益なことで...,特定社会保険労務士は、個別労働紛争について厚生労働大臣が指定する団体が行う裁判外紛争解決手...,社会保険労務士法第２条及び同法第２条の２,対応不可,平成26年の社会保険労務士法改正により、社会保険労務士の業務の範囲が広がったところであり、...,規制改革,規4-0004,,UID00003
4,5,2022-06-29,2022-07-22,弁護士等による戸籍謄本等の職務上請求書について、職印の押印を廃止すること,個人,法務省,NaN,NaN,弁護士等による戸籍謄本等の職務上請求書について、職印の押印を必要とする戸籍法施行規則の規定を...,近年の押印廃止の流れにより、一般国民や官公署が戸籍謄本を請求する場合は押印を要しなくなったが...,弁護士等が、戸籍法第１０条の２第３項から第５項までの規定に基づき戸籍謄本等を請求する場合には...,戸籍法第１０条の２第３項～第５項\n戸籍法第１０条の３第１項\n戸籍法施行規則第１１条の２第４号,対応不可,統一請求書の使用及び当該統一請求書に係る弁護士等の職印の押印については、不正請求を防止する観...,規制改革,規4-0005,デジタル\nハンコ,UID00004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3789,1030,2021-08-29,2021-11-06,政府統計のローデータをSQLにて公開する,個人,総務省,NaN,NaN,\n政府統計全般のローデータを（個人情報保護の処理は行ったうえで）SQLサーバにて公開し、広...,\n現在の政府統計は形式がバラバラであるが、もしこれを仮にCSV等の形式としてe-Stadで...,統計を作成するために用いられる調査票情報（ローデータ）については、個人又は法人その他の団体の...,なし,現行制度下で対応可能,制度の現状欄に記載のとおりです。,行政改革,行2-1030,デジタル,UID03789
3790,1031,2021-08-29,NaT,文書作成時、スペースを使った均等割り付けの禁止,個人,デジタル庁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行2-1031,,UID03790
3791,1032,2021-08-29,NaT,国家資格の免許証のカード化,個人,デジタル庁\n総務省\n警察庁\n金融庁\n消費者庁\n法務省\n財務省\n文部科学省\n厚...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行2-1032,,UID03791
3792,1033,2021-09-04,2021-11-06,土地に関して、様々な組織が管理する情報を統合して管理すべき（縦割りの解消）,個人,国土交通省\n法務省,NaN,NaN,\n土地の現状に関して主に公的機関が関する情報（登記に関する地図（公図）、国土地理院の地図、...,\n土地の権利の確認は、まず登記記録の確認から始まる。公図は不整合のものが大変多く、実際の土...,【法務省】\n各官署において，所管する制度等に応じ，必要な情報を保有しているものと承知してい...,測量法第27条第2項、道路法第28条、土地収用法第47条の2第1項、第66条第3項、下水道法...,対応不可,【法務省】\n 制度の現状欄に記載のとおり，各官署において所管する制度等に応じて当該制度に係...,行政改革,行2-1033,デジタル,UID03792
